# Build a Local RAG Application

## Document Loading

In [9]:
pip install --quiet gpt4all


[notice] A new release of pip is available: 23.0.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install --quiet langchain-community beautifulsoup4 langchain-chroma


[notice] A new release of pip is available: 23.0.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [25]:
pip install --quiet langchain-huggingface

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.0.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [26]:
# added/edited
from langchain_huggingface import HuggingFaceEmbeddings

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)


from langchain_chroma import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings

# Require GPU
# vectorstore = Chroma.from_documents(documents=all_splits, embedding=GPT4AllEmbeddings(model_name="all-MiniLM-L6-v2.gguf2.f16.gguf"))

# added/edited
vectorstore = Chroma.from_documents(documents=all_splits, embedding=HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2"))

question = "What are the approaches to Task Decomposition?"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [8]:
docs[0]

Document(page_content='Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.', metadata={'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en', 'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log"})

## Model

### LLaMA2

In [4]:
pip install --quiet llama-cpp-python


[notice] A new release of pip is available: 23.0.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
! CMAKE_ARGS="-DLLAMA_METAL=on" FORCE_CMAKE=1 /Users/rlm/miniforge3/envs/llama/bin/pip install -U llama-cpp-python --no-cache-dir

/bin/bash: line 1: /Users/rlm/miniforge3/envs/llama/bin/pip: No such file or directory


In [ ]:
from langchain_community.llms import LlamaCpp


n_gpu_layers = 1  # Metal set to 1 is enough.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="/Users/rlm/Desktop/Code/llama.cpp/models/llama-2-13b-chat.ggufv3.q4_0.bin",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=2048,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    verbose=True,
)

In [13]:
from llama_cpp import Llama

llm = Llama.from_pretrained(
    repo_id="TheBloke/Llama-2-13B-chat-GGUF",
    filename="llama-2-13b-chat.Q4_0.gguf",
    verbose=False
)

In [ ]:
response = llm.create_chat_completion(
    messages = [
        {
            "role": "user",
            "content": [
                {"type" : "text", "text": "Simulate a rap battle between Stephen Colbert and John Oliver"}
            ]
        }
    ]
)
print(response)

In [ ]:
response['choices'][0]['message']['content']

### GPT4All

In [ ]:
from langchain_community.llms import GPT4All

gpt4all = GPT4All(
    model="nous-hermes-llama2-13b.Q4_0.gguf",
    max_tokens=2048,
)

### llamafile

In [10]:
from langchain_community.llms.llamafile import Llamafile

llamafile = Llamafile()

llamafile.invoke("Here is my grandmother's beloved recipe for spaghetti and meatballs:")

'\n\nIngredients:\n- 1 pound ground beef (ground sirloin or ground round)\n- 1/4 cup chopped fresh parsley\n- 2 tbsp olive oil\n- 1 onion, chopped\n- 3 garlic cloves, minced\n- 8 cups whole wheat spaghetti (or regular spaghetti)\n- 1 can crushed tomatoes (15.25 oz)\n- Salt and pepper to taste\n- Freshly ground black pepper to taste\n- 1 egg\n- Cooked and shredded mozzarella cheese (3 oz)\n- Parmesan cheese (3 oz) for garnish\n- Whole wheat bread for serving\n\nInstructions:\n\n1. Preheat the oven to 375°F.\n\n2. In a large bowl, mix together the ground beef, parsley, olive oil, onion, minced garlic, crushed tomatoes (you can use chicken or vegetable), salt, pepper, and egg until well combined.\n\n3. Divide the meat mixture into 12 equal portions, shaping each portion with your hands into a meatball shape.\n\n4. Place the meatballs on a baking sheet lined with parchment paper.\n\n5. Bake in the preheated oven for 20-25 minutes or until cooked through.\n\n6. While the meatballs are cooki

## Using in a chain

In [11]:
# added/edited
llm = llamafile

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

# Prompt
prompt = PromptTemplate.from_template(
    "Summarize the main themes in these retrieved docs: {docs}"
)


# Chain
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = {"docs": format_docs} | prompt | llm | StrOutputParser()

# Run
question = "What are the approaches to Task Decomposition?"
docs = vectorstore.similarity_search(question)
chain.invoke(docs)

' The process of task decomposition can be even more challenging. Human inputs are essential in tasks with multiple subgoals that require coordination, such as creating a novel, and where the goal is to combine different pieces of information into meaningful insights.\n\nCase studies:\n1) Using LLMs for planning and decomposing complex tasks like "Write a story" or "Create a poem." in video games. For example, Google\'s T5 model was used to plan and generate storylines for the game "Minecraft Story Mode." In this task, the model uses a simple prompt of "Write a story about [character]" and then generates a plot by answering questions like "What happens next?" or "What happened at that point?"\n2) Using LLMs for planning and decomposing complex tasks in medical diagnoses, such as "Diagnose and treat a patient with [disease]." The model uses the task as a prompt to generate relevant diagnostic questions and then uses machine learning techniques like clustering or graph analysis to decomp

## Q&A

In [17]:
pip install --quiet langchainhub

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.0.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
from langchain import hub

rag_prompt = hub.pull("rlm/rag-prompt")
rag_prompt.messages

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]

In [19]:
from langchain_core.runnables import RunnablePassthrough, RunnablePick

# Chain
chain = (
    RunnablePassthrough.assign(context=RunnablePick("context") | format_docs)
    | rag_prompt
    | llm
    | StrOutputParser()
)

# Run
chain.invoke({"context": docs, "question": question})

' \nTask decompositions can be done using various techniques, such as the first method (simple prompting) or the third method (human inputs). In this case, human intervention is essential to ensure the task decomposition process is efficient and effective.</s>'

In [20]:
# Prompt
rag_prompt_llama = hub.pull("rlm/rag-prompt-llama")
rag_prompt_llama.messages

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="[INST]<<SYS>> You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.<</SYS>> \nQuestion: {question} \nContext: {context} \nAnswer: [/INST]"))]

In [21]:
# Chain
chain = (
    RunnablePassthrough.assign(context=RunnablePick("context") | format_docs)
    | rag_prompt_llama
    | llm
    | StrOutputParser()
)

# Run
chain.invoke({"context": docs, "question": question})

'<<SYS>> The approach to task decomposition is based on the concept of story outlines. Tasks are decomposed into subgoals, and these subgoals can be planned in a sequence-like way. However, LLMs struggle with dealing with long histories, as they cannot remember everything from scratch. Human planners can adjust plans when faced with unexpected errors, while LLMs struggle to do so due to the memory constraints of their models.</s>'

## Q&A with retrieval

In [22]:
question = "What are the approaches to Task Decomposition?"

In [23]:
retriever = vectorstore.as_retriever()
qa_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)


qa_chain.invoke(question)

'\nLong-term planning refers to the process of identifying and anticipating future situations. Task decomposition is a technique used in long-term planning to identify subgoals that can be addressed by different tasks. The approach to task decomposition depends on the problem at hand, with some using simple prompting or instructions while others use human input. Long-term planning requires effective planning over a lengthy history and efficient exploration of the solution space.</s>'